In [3]:
import os
import numpy as np
import pandas as pd
import joblib
import json
import tensorflow as tf
import gc

2021-09-20 09:37:30.511722: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/lib
2021-09-20 09:37:30.511815: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
with open('../tunings/TrigL2_20180125_v8/ElectronRingerLooseTriggerConfig.json') as f:
    conf = json.load(f)

In [5]:
conf

{'__name__': 'TrigL2_20180125_v8',
 '__version__': 'v8',
 '__operation__': 'Loose',
 '__signature__': 'electron',
 'Model__size': 25,
 'Model__etmin': [15.0,
  15.0,
  15.0,
  15.0,
  15.0,
  20.0,
  20.0,
  20.0,
  20.0,
  20.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  40.0,
  40.0,
  40.0,
  40.0,
  40.0,
  50.0,
  50.0,
  50.0,
  50.0,
  50.0],
 'Model__etmax': [20.0,
  20.0,
  20.0,
  20.0,
  20.0,
  30.0,
  30.0,
  30.0,
  30.0,
  30.0,
  40.0,
  40.0,
  40.0,
  40.0,
  40.0,
  50.0,
  50.0,
  50.0,
  50.0,
  50.0,
  500000.0,
  500000.0,
  500000.0,
  500000.0,
  500000.0],
 'Model__etamin': [0.0,
  0.8,
  1.37,
  1.54,
  2.37,
  0.0,
  0.8,
  1.37,
  1.54,
  2.37,
  0.0,
  0.8,
  1.37,
  1.54,
  2.37,
  0.0,
  0.8,
  1.37,
  1.54,
  2.37,
  0.0,
  0.8,
  1.37,
  1.54,
  2.37],
 'Model__etamax': [0.8,
  1.37,
  1.54,
  2.37,
  2.5,
  0.8,
  1.37,
  1.54,
  2.37,
  2.5,
  0.8,
  1.37,
  1.54,
  2.37,
  2.5,
  0.8,
  1.37,
  1.54,
  2.37,
  2.5,
  0.8,
  1.37,
  1.54,
  2.37,
  2.

In [45]:
def open_boosted(path= ""):
    '''
    This function will get a .npz file and transform into a pandas DataFrame. 
    The .npz has three types of data: float, int and bool this function will concatenate these features and reorder them.

    Arguments:
    path (str) - the full path to .npz file
    '''
    # open the file
    d = dict(np.load(path, allow_pickle=True))   
    #print(d.keys()) 
    # create a list of temporary dataframes that should be concateneted into a final one
    df_list = []
    for itype in ['float', 'int', 'bool', 'object']:
        df_list.append(pd.DataFrame(data=d['data_%s' %itype], columns=d['features_%s' %itype]))
    # concat the list
    df = pd.concat(df_list, axis=1)
    # return the DataFrame with ordered features.
    df = df[d['ordered_features']]
    # add the target information
    df['target'] = d['target']
    df['et_bin'] = d['etBinIdx']
    df['eta_bin'] = d['etaBinIdx']
    # remove the list of DataFrame and collect into garbage collector
    del df_list, d
    gc.collect()
    return df

In [111]:
data_path = '/media/natmourajr/Backup/Work/CERN/Qualify/cern_data/Zee_boosted/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins/mc16_13TeV.302236_309995_341330.sgn.boosted_probes.WZ_llqq_plus_radion_ZZ_llqq_plus_ggH3000.merge.25bins_et0_eta0.npz'
df = open_boosted(data_path)

In [106]:
df.head()

,RunNumber,avgmu,trig_L2_cl_et,trig_L2_cl_eta,trig_L2_cl_phi,trig_L2_cl_reta,trig_L2_cl_ehad1,trig_L2_cl_eratio,trig_L2_cl_f1,trig_L2_cl_f3,...,trig_L2_cl_lhtight_et0to12,trig_L2_cl_lhtight_et12to20,trig_L2_cl_lhtight_et22toInf,trig_L2_el_cut_pt0to15,trig_L2_el_cut_pt15to20,trig_L2_el_cut_pt20to50,trig_L2_el_cut_pt50toInf,target,et_bin,eta_bin
0,300000,25.490000,18339.359375,-0.029264,1.975188,0.988089,-184.739105,0.816197,0.085186,0.008879,...,True,True,True,True,True,True,True,1.0,0,0
1,300000,44.490002,19970.226562,0.278382,-2.358062,0.935216,-161.464890,0.890477,0.466089,-0.000920,...,True,True,True,True,True,True,True,1.0,0,0
2,300000,44.490002,17113.560547,0.618808,1.590098,0.962935,-130.190170,0.948033,0.320025,0.001840,...,True,True,True,True,True,True,True,1.0,0,0
3,300000,27.500000,18358.531250,-0.192658,1.274844,0.670090,5028.686035,0.138930,0.270767,0.011920,...,False,False,False,True,True,True,True,1.0,0,0
4,300000,34.490002,18919.300781,-0.027433,1.948146,0.949287,-222.559723,0.924114,0.328973,0.002532,...,True,True,True,True,True,True,True,1.0,0,0


In [112]:
def add_tuning_decision(df, config_dict):
        #find et and eta bin
        idx = (df['et_bin']+df['eta_bin']).unique()[0]
        print(idx)

        m_path = "/home/natmourajr/Workspace/UFRJ/CERN/CERN-ATLAS-Qualify/analysis/tunings/TrigL2_20180125_v8"
        m_model_path = os.path.join(m_path,config_dict["Model__path"][idx].replace('.onnx',''))
        with open(m_model_path + '.json', 'r') as f:
                sequential = json.load(f) 
        model = tf.keras.models.model_from_json(json.dumps(sequential, separators=(',', ':')))
        model.load_weights(m_model_path + '.h5')
        slope, offset = config_dict["Threshold__slope"][idx],config_dict["Threshold__offset"][idx]

        # ring list
        rings = ['trig_L2_cl_ring_%i' %(iring) for iring in range(100)] 

        def norm1( data ): 
                norms = np.abs( data.sum(axis=1) )
                norms[norms==0] = 1
                return data/norms[:,None] 
        df['nn_output_%s_%s'%(config_dict['__operation__'].lower(),config_dict['__version__'])] = model.predict(norm1(df[rings].values)) 
        df['thr_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])] = df['avgmu']*slope + offset
        df['nn_decision_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])] = 0
        df.loc[df['nn_output_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])] > df['thr_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])],'nn_decision_%s_%s' %(config_dict['__operation__'].lower(),config_dict['__version__'])] = 1 


In [113]:
add_tuning_decision(df=df, config_dict=conf)

0


In [114]:
df.head()

,RunNumber,avgmu,trig_L2_cl_et,trig_L2_cl_eta,trig_L2_cl_phi,trig_L2_cl_reta,trig_L2_cl_ehad1,trig_L2_cl_eratio,trig_L2_cl_f1,trig_L2_cl_f3,...,trig_L2_el_cut_pt0to15,trig_L2_el_cut_pt15to20,trig_L2_el_cut_pt20to50,trig_L2_el_cut_pt50toInf,target,et_bin,eta_bin,nn_output_loose_v8,thr_loose_v8,nn_decision_loose_v8
0,300000,25.490000,18339.359375,-0.029264,1.975188,0.988089,-184.739105,0.816197,0.085186,0.008879,...,True,True,True,True,1.0,0,0,0.085557,-0.339945,1
1,300000,44.490002,19970.226562,0.278382,-2.358062,0.935216,-161.464890,0.890477,0.466089,-0.000920,...,True,True,True,True,1.0,0,0,1.950747,-0.535281,1
2,300000,44.490002,17113.560547,0.618808,1.590098,0.962935,-130.190170,0.948033,0.320025,0.001840,...,True,True,True,True,1.0,0,0,4.567322,-0.535281,1
3,300000,27.500000,18358.531250,-0.192658,1.274844,0.670090,5028.686035,0.138930,0.270767,0.011920,...,True,True,True,True,1.0,0,0,-5.566738,-0.360610,0
4,300000,34.490002,18919.300781,-0.027433,1.948146,0.949287,-222.559723,0.924114,0.328973,0.002532,...,True,True,True,True,1.0,0,0,3.920913,-0.432472,1
